#### 神经网络的Pytorch API

> torch.nn.Linear(in_features,bias=True,device=None,dtype=None)

Applies a linar transformation to the incoming data: $y=xA^T+b$

Parameters:
- in_features(int), out_features(int): size
- bias(bool): True *初始化随机数，可随训练改变*, false *固定*
- dtype: 数据类型

损失函数(loss)：预测值和真值的差异  ${\rm loss} = \frac{1}{2} \left(y-\hat{y}\right)^2$

In [ ]:
from torch.utils.data import Dataset
import os
import pandas as pd # 加载表格
import numpy as np
import torch

class iris_dataloader(Dataset):
	def __init__(self, data_path):
		super().__init__()
		self.data_path = data_path
		assert os.path.exists(self.data_path)
		
		df = pd.read_csv(self.data_path, names=[0,1,2,3,4])
		d = {"Iris-setosa":0, "Iris-versicolor":1, "Iris-virginica":2}
		df[4] = df[4].map(d)

		data = df.iloc[:, 0:4]
		label = df.iloc[:, 4]
		data = (data - np.mean(data) / np.std(data)) # 归一化(Z值化)
		self._data = torch.from_numpy(np.array(data, dtype='float32'))
		self._label = torch.from_numpy(np.array(label, dtype='int64'))
		self._data_num = len(label)

		self.data = list(self._data)
		self.label = list(self._label)

	def __len__(self):
		return self._data_num

	def __getitem__(self, index): # 获取并返回一个数据样本
		return self.data[index], self.label[index] ## That is Different from Lecture

In [26]:
# 初始化神经网络模型
import torch.nn as nn
from tqdm import tqdm

class NN(torch.nn.Module):
	def __init__(self, in_dim, hid_dim1, hid_dim2, out_dim):
		super().__init__()
		self.layer1 = nn.Linear(in_dim, hid_dim1)
		self.layer2 = nn.Linear(hid_dim1, hid_dim2)
		self.layer3 = nn.Linear(hid_dim2, out_dim)

	def forward(self, x):
		x = self.layer1(x)
		x = x.relu()
		x = self.layer2(x)
		x = x.relu()
		x = self.layer3(x)
		return x

In [27]:
# 定义计算环境 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [28]:
# 训练集、验证集和测试集
import torch.utils
import torch.utils.data
from torch.utils.data import DataLoader

custom_dataset = iris_dataloader('./Iris.data.txt')
train_size = int(len(custom_dataset) * 0.7)
val_size   = int(len(custom_dataset) * 0.2)
test_size  = len(custom_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = \
torch.utils.data.random_split(custom_dataset,[train_size,val_size,test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
val_loader   = DataLoader(val_dataset,   batch_size=1,  shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=1,  shuffle=False)
print(f"训练集的大小: {len(train_loader)*16}")
print(f"验证集的大小: {len(val_loader)}")
print(f"测试集的大小: {len(test_loader)}")

训练集的大小: 112
验证集的大小: 30
测试集的大小: 15


c:\Users\Admin\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


In [ ]:
# 定义一个推理函数，来计算并返回准确率
import sys
import torch.optim as optim
def infer(model: torch.nn.Module, dataset, device): 
	model.eval() # 将模型置于验证状态下
	acc_num = 0
	with torch.no_grad(): # 上下文管理器
		# Context-manager that disables gradient calculation.
		for data in dataset:
			datas, label = data
			outputs = model(datas.to(device))
			predict_y = torch.max(outputs, dim=1)[1]
			acc_num += torch.eq(predict_y, label.to(device)).sum().item() # 对每一批数量进行累加
	acc = acc_num / len(dataset)
	return acc

def main(lr = 0.005, epochs = 20):
	model = NN(4, 12, 6, 3).to(device) # 最后的分类数
	loss_f = nn.CrossEntropyLoss()

	pg = [p for p in model.parameters() if p.requires_grad] # 列表生成式
	optimizer = optim.Adam(pg, lr=lr) # 可训练的参数

	# 权重的文件存储路径
	save_path = os.path.join(os.getcwd(), "result/weights")
	if os.path.exists(save_path) is False:
		os.makedirs(save_path)

	# 开始训练
	for epoch in range(epochs):
		model.train()
		acc_num = torch.zeros(1).to(device)
		sample_num = 0
		train_bar = tqdm(train_loader, file=sys.stdout, ncols=100)
		for datas in train_bar:
			data, label = datas
			label = label.squeeze(-1)
			sample_num += data.shape[0]
			
			optimizer.zero_grad()
			outputs = model(data.to(device))
			pred_class = torch.max(outputs, dim=1)[1] # torch.max 返回的是元组，[0]为大小 [1]为索引
			acc_num = torch.eq(pred_class, label.to(device)).sum()
			
			loss = loss_f(outputs, label.to(device))
			loss.backward() # 梯度求导
			optimizer.step()

			train_acc = acc_num / sample_num
			train_bar.desc = f'Train epoch[{epoch+1}/{epochs}] \
				ACC:{train_acc:.3f} Loss:{loss:.3f}'
		
		# 在每一轮训练后进行验证
		val_acc = infer(model, val_loader, device)
		print(f'Train epoch[{epoch+1}/{epochs}] \
			Train ACC:{train_acc:.3f} Value ACC:{val_acc:.3f}')
		torch.save(model.state_dict(), os.path.join(save_path, "nn.pth"))

		# 每次数据集迭代后，要对初始化的指标清零
		train_acc = 0.0
		val_acc   = 0.0
	print('Finish Training')

	test_acc = infer(model, test_loader, device)
	print(f'Test ACC: {test_acc}')

if __name__ == '__main__':
	main()